In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
import sklearn

In [2]:
df = pd.read_csv("E:/Courses/Machin Learning/Machine_Learning1-faradars/Files_and_codes/S05/code-5/train_Titanic.csv")
df.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1


In [3]:
df_test = pd.read_csv("E:/Courses/Machin Learning/Machine_Learning1-faradars/Files_and_codes/S05/code-5/test_Titanic.csv")      
df_test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [4]:
#Preprocessing

In [5]:
def f(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'


def g(tt):
    if tt in ['Mr']:
        return 1
    elif tt in ['Master']:
        return 3
    elif tt in ['Ms', 'Mlle', 'Miss']:
        return 4
    elif tt in ['Mrs','Mme']:
        return 5
    else:
        return 2


df['title'] = df['Name'].apply(f).apply(g)

df_test['title'] = df_test['Name'].apply(f).apply(g)

# drop unnecessary columns
df = df.drop(['PassengerId','Name','Ticket'], axis=1)

df_test    = df_test.drop(['Name','Ticket'], axis=1)   

##
edt  = pd.get_dummies(df['Embarked'])
edt.drop(['S'], axis=1, inplace=True)
edt_test  = pd.get_dummies(df_test['Embarked'])
edt_test.drop(['S'], axis=1, inplace=True)
df = df.join(edt)
df_test = df_test.join(edt_test)
df.drop(['Embarked'], axis=1,inplace=True)
df_test.drop(['Embarked'], axis=1,inplace=True)

## Fare
df_test["Fare"].fillna(df_test["Fare"].median(), inplace=True)


# Age impute
df['Age']      = df.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))
df_test['Age'] = df_test.groupby(['Pclass'])['Age'].transform(lambda x: x.fillna(x.mean()))


# Cabin
# It has a lot of NaN values, so it won't cause a remarkable impact on prediction
df.drop("Cabin",axis=1,inplace=True)
df_test.drop("Cabin",axis=1,inplace=True)


##sexes
s = sorted(df['Sex'].unique())
z=zip(s, range(0, len(s) + 1))
gm = dict(z)
df['Sex'].head(5)
df['Sex'] = df['Sex'].map(gm).astype(int)
df['Sex'].head(5)
df_test['Sex'] = df_test['Sex'].map(gm).astype(int)


In [6]:
df.head(7)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived,title,C,Q
0,3,1,22.00000,1,0,7.2500,0,1,0,0
1,1,0,38.00000,1,0,71.2833,1,5,1,0
2,3,0,26.00000,0,0,7.9250,1,4,0,0
3,1,0,35.00000,1,0,53.1000,1,5,0,0
4,3,1,35.00000,0,0,8.0500,0,1,0,0
5,3,1,25.14062,0,0,8.4583,0,1,0,1
6,1,1,54.00000,0,0,51.8625,0,1,0,0


In [7]:
# define training and testing sets

In [8]:
X_train = df.drop("Survived",axis=1)  
Y_train = df["Survived"]

In [9]:
model =  LogisticRegression()

In [10]:
%%capture
model.fit(X_train, Y_train)

In [11]:
model.score(X_train, Y_train)

0.8249158249158249

In [12]:
X_test  = df_test.drop("PassengerId",axis=1).copy()

In [13]:
Y_pred = model.predict(X_test)

In [14]:
dfo = pd.DataFrame({ "Id": df_test["PassengerId"], "Survived": Y_pred })

In [15]:
dfo.to_csv('E:/Courses/Machin Learning/Machine_Learning1-faradars/Files_and_codes/S05/code-5/titanic-out.csv',index=False)